## Introduction

In this example, we train a model to learn to add two numbers, provided as strings.

**Example:**

- Input: "535+61"
- Output: "596"

Input may optionally be reversed, which was shown to increase performance in many tasks
 in: [Learning to Execute](http://arxiv.org/abs/1410.4615) and
[Sequence to Sequence Learning with Neural Networks](http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf).

Theoretically, sequence order inversion introduces shorter term dependencies between
 source and target for this problem.

**Results:**

For two digits (reversed):

+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs

Three digits (reversed):

+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs

Four digits (reversed):

+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs

Five digits (reversed):

+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs

14 digits (reversed):

+ Three layers LSTM (128 HN), 700k training examples = 99% val accuracy in 25 epochs

21 digits(reversed):

+ Three layers LSTM (128 HN), 800k training examples = 79.5% val accuracy in 30 epochs

21 digits(reversed):

+ Five layers LSTM (128 HN), 800k training examples = 76.7% val accuracy in 20 epochs (same as for three layers in 20 epochs)

30 digits(reversed):

+ Three layers LSTM (128 HN), 1kk training examples = 77.5% val accuracy in 25 epochs

You can try to increase the digits to the desired number and train if necessary, but this will take a lot of time.



## Setup

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model("rnn_for_add_14_700k.keras")

#YOU SHUOLD WRITE `DIGITS` LIKE IN NAME OF CHOSEN MODEL
DIGITS = 14
MAXLEN = DIGITS + 1 + DIGITS

## Generate the data

In [ ]:
class CharacterTable:
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """

    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[x] for x in x)


# All the numbers, plus sign and space for padding.
chars = "0123456789+ "
ctable = CharacterTable(chars)

In [ ]:
import random
maxint = 10**13
TESTSIZE = 100
#HERE YOU CAN USE YOUR OWN TEST DATASET - LIST OF TUPLES
#EXAMPLE: q_list = [(2, 2), (456, 234), (5674443, 234234)]
q_list = []

#GENERATION TEST DATASET NUMS
for i in range(TESTSIZE):
    q_list.append((random.randint(1, maxint), random.randint(1, maxint)))

quests = []
expects = []

for elem in q_list:
    a, b = elem[0], elem[1]
    q = "{}+{}".format(a, b)
    query = q + " " * (MAXLEN - len(q))
    query = query[::-1]
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += " " * (DIGITS + 1 - len(ans))
    expects.append(ans)
    quests.append(query)

print("Vectorization...")
x = np.zeros((len(quests), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(quests), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(quests):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expects):
    y[i] = ctable.encode(sentence, DIGITS + 1)


Vectorization...


<ipython-input-20-9d343d704976>:23: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(quests), MAXLEN, len(chars)), dtype=np.bool)
<ipython-input-20-9d343d704976>:24: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(quests), DIGITS + 1, len(chars)), dtype=np.bool)


In [ ]:
score = model.evaluate(x, y, verbose = 0)
print('Test accuracy:', score[1])

Test accuracy: 0.9126666784286499


In [ ]:
for i in range(len(quests)):
    rowx, rowy = x[np.array([i])], y[np.array([i])]
    preds = np.argmax(model.predict(rowx), axis=-1)
    correct = ctable.decode(rowy[0])
    guess = ctable.decode(preds[0], calc_argmax=False)
    if i % 10 == 0:
      print("Q", q[::-1])
      print("T", correct, end=" ")
      if correct == guess:
          print("☑ " + guess)
      else:
          print("☒ " + guess)

1/1 [==============================] - 0s 36ms/step
Q 7449745059291+557608037711
T 8975650461966   ☑ 8975650461966  
1/1 [==============================] - 0s 31ms/step
Q 7449745059291+557608037711
T 4577350733454   ☒ 4577350743454  
1/1 [==============================] - 0s 43ms/step
Q 7449745059291+557608037711
T 1914771418972   ☑ 1914771418972  
1/1 [==============================] - 0s 41ms/step
Q 7449745059291+557608037711
T 15867546546567  ☒ 15867546646576 
1/1 [==============================] - 0s 31ms/step
Q 7449745059291+557608037711
T 10465079052032  ☑ 10465079052032 
1/1 [==============================] - 0s 29ms/step
Q 7449745059291+557608037711
T 12338139383020  ☑ 12338139383020 
1/1 [==============================] - 0s 32ms/step
Q 7449745059291+557608037711
T 4676572808989   ☒ 4676571808989  
1/1 [==============================] - 0s 36ms/step
Q 7449745059291+557608037711
T 3076466827498   ☒ 3066466927498  
1/1 [==============================] - 0s 32ms/step
Q 7449745059